# 构建一个聊天机器人

:::note

本教程之前使用了 [RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html) 抽象。您可以在 [v0.2 文档](https://python.langchain.com/v0.2/docs/tutorials/chatbot/) 中访问该版本的文档。

从 LangChain 的 v0.3 版本开始，我们建议用户利用 [LangGraph 持久化](https://langchain-ai.github.io/langgraph/concepts/persistence/) 将 `memory` 集成到新的 LangChain 应用中。

如果您的代码已经依赖于 `RunnableWithMessageHistory` 或 `BaseChatMessageHistory`，您无需进行任何更改。我们近期没有计划弃用此功能，因为它适用于简单的聊天应用，任何使用 `RunnableWithMessageHistory` 的代码将继续正常工作。

请参阅 [如何迁移到 LangGraph Memory](/docs/versions/migrating_memory/) 了解更多详细信息。
:::

## 概述

我们将通过一个示例来学习如何设计和实现一个基于 LLM 的聊天机器人。
这个聊天机器人将能够与 [聊天模型](/docs/concepts/chat_models) 进行对话并记住之前的交互。

请注意，我们构建的这个聊天机器人仅使用语言模型进行对话。
您可能正在寻找以下几个相关概念：

- [对话式 RAG](/docs/tutorials/qa_chat_history)：在外部数据源上启用聊天机器人体验
- [代理](/docs/tutorials/agents)：构建可以执行操作的聊天机器人

本教程将涵盖基础知识，这些知识对于上述两个更高级的主题非常有用，但如果您愿意，可以直接跳到那里。

## 设置

### Jupyter Notebook

本指南（以及文档中的大多数其他指南）使用 [Jupyter notebooks](https://jupyter.org/) 并假设读者也在使用。Jupyter notebooks 非常适合学习如何使用 LLM 系统，因为有时可能会出现问题（意外输出、API 停止工作等），在交互式环境中学习指南是更好地理解它们的好方法。

本教程和其他教程可能最方便在 Jupyter notebook 中运行。请参阅 [此处](https://jupyter.org/install) 了解安装说明。

### 安装

对于本教程，我们需要 `langchain-core` 和 `langgraph`。本指南需要 `langgraph >= 0.2.28`。

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install langchain-core langgraph>0.2.27</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain-core langgraph>0.2.27 -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>



有关更多详细信息，请参阅我们的 [安装指南](/docs/how_to/installation)。

### LangSmith

您使用 LangChain 构建的许多应用程序将包含多个步骤和多次 LLM 调用。
随着这些应用程序变得越来越复杂，能够检查链或代理内部发生的确切情况变得至关重要。
最好的方法是使用 [LangSmith](https://smith.langchain.com)。

在上面的链接注册后，请确保设置环境变量以开始记录跟踪：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

或者，如果在 notebook 中，您可以使用以下方式设置它们：

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

## 快速入门

首先，让我们学习如何单独使用语言模型。LangChain 支持许多不同的语言模型，您可以互换使用它们 - 选择您想要使用的模型！

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4o-mini"}}} />


In [2]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

首先直接使用模型。`ChatModel` 是 LangChain "Runnables" 的实例，这意味着它们提供了一个标准接口来与之交互。要简单地调用模型，我们可以将消息列表传递给 `.invoke` 方法。

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="你好！我是 Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-5211544f-da9f-4325-8b8e-b3d92b2fc71a-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

模型本身没有任何状态的概念。例如，如果您问一个后续问题：

In [ ]:
model.invoke([HumanMessage(content="我的名字是什么？")])

AIMessage(content="I'm sorry, but I don't have access to personal information about users unless it has been shared with me in the course of our conversation. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 11, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-a2d13a18-7022-4784-b54f-f85c097d1075-0', usage_metadata={'input_tokens': 11, 'output_tokens': 34, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

让我们看看示例 [LangSmith 跟踪](https://smith.langchain.com/public/5c21cb92-2814-4119-bae9-d02b8db577ac/r)

我们可以看到它没有将之前的对话轮次纳入上下文，无法回答问题。
这会导致糟糕的聊天机器人体验！

为了解决这个问题，我们需要将整个 [对话历史](/docs/concepts/chat_history) 传递给模型。让我们看看这样做会发生什么：

In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="你好！我是 Bob"),
        AIMessage(content="你好 Bob！今天我能为您做些什么？"),
        HumanMessage(content="我的名字是什么？"),
    ]
)

AIMessage(content='Your name is Bob! How can I help you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 33, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-34bcccb3-446e-42f2-b1de-52c09936c02c-0', usage_metadata={'input_tokens': 33, 'output_tokens': 14, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

现在我们可以看到我们得到了一个好的响应！

这是聊天机器人能够进行对话的基本理念。
那么我们如何最好地实现这一点呢？

## 消息持久化

[LangGraph](https://langchain-ai.github.io/langgraph/) 实现了一个内置的持久化层，非常适合支持多轮对话的聊天应用。

将我们的聊天模型包装在一个最小的 LangGraph 应用中，可以自动持久化消息历史，简化多轮应用的开发。

LangGraph 配备了一个简单的内存检查点，我们在下面使用它。请参阅其 [文档](https://langchain-ai.github.io/langgraph/concepts/persistence/) 了解更多详细信息，包括如何使用不同的持久化后端（例如 SQLite 或 Postgres）。

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# 定义一个新的图
workflow = StateGraph(state_schema=MessagesState)


# 定义调用模型的函数
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# 定义图中的（单个）节点
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# 添加内存
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

我们现在需要创建一个 `config`，每次传递给 runnable。此配置包含不是直接输入的一部分但仍然有用的信息。在本例中，我们希望包括一个 `thread_id`。这应该看起来像：

In [7]:
config = {"configurable": {"thread_id": "abc123"}}

这使我们能够通过单个应用支持多个对话线程，这是当您的应用有多个用户时的常见需求。

然后我们可以调用应用程序：

In [ ]:
query = "你好！我是 Bob。"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # 输出包含状态中的所有消息

================================== Ai Message ==================================

Hi Bob! How can I assist you today?


In [ ]:
query = "我的名字是什么？"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! How can I help you today, Bob?


太棒了！我们的聊天机器人现在记得关于我们的信息。如果我们更改配置以引用不同的 `thread_id`，我们可以看到它重新开始对话。

In [10]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about you unless you've shared it in this conversation. How can I assist you today?


但是，我们可以随时回到原始对话（因为我们将其持久化在数据库中）

In [11]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. What would you like to discuss today?


这就是我们如何支持聊天机器人与许多用户进行对话的方式！

:::tip

对于异步支持，将 `call_model` 节点更新为异步函数，并在调用应用程序时使用 `.ainvoke`：

```python
# 节点的异步函数：
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# 像之前一样定义图：
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# 异步调用：
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()
```

:::

现在，我们所做的只是围绕模型添加了一个简单的持久化层。我们可以通过添加提示模板使聊天机器人更加复杂和个性化。

## 提示模板

[提示模板](/docs/concepts/prompt_templates) 有助于将原始用户信息转换为 LLM 可以处理的格式。在本例中，原始用户输入只是一个消息，我们将其传递给 LLM。现在让我们使其稍微复杂一些。首先，我们添加一个带有一些自定义指令的系统消息（但仍然将消息作为输入）。接下来，我们将添加除消息之外的更多输入。

要添加系统消息，我们将创建一个 `ChatPromptTemplate`。我们将利用 `MessagesPlaceholder` 来传递所有消息。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你像海盗一样说话。尽最大努力回答所有问题。",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

我们现在可以更新我们的应用程序以合并此模板：

In [13]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    # highlight-start
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    # highlight-end
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

我们以相同的方式调用应用程序：

In [ ]:
config = {"configurable": {"thread_id": "abc345"}}
query = "你好！我是 Jim。"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Jim! What brings ye to these waters today? Be ye seekin' treasure, knowledge, or perhaps a good tale from the high seas? Arrr!


In [ ]:
query = "我的名字是什么？"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ye be called Jim, matey! A fine name fer a swashbuckler such as yerself! What else can I do fer ye? Arrr!


太棒了！现在让我们的提示稍微复杂一些。假设提示模板现在看起来像这样：

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个乐于助人的助手。用 {language} 尽最大努力回答所有问题。",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

请注意，我们已向提示添加了一个新的 `language` 输入。我们的应用程序现在有两个参数——输入 `messages` 和 `language`。我们应该更新应用程序的状态以反映这一点：

In [20]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


# highlight-next-line
class State(TypedDict):
    # highlight-next-line
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # highlight-next-line
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc456"}}
query = "你好！我是 Bob。"
language = "西班牙语"

input_messages = [HumanMessage(query)]
output = app.invoke(
    # highlight-next-line
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob! ¿Cómo puedo ayudarte hoy?


请注意，整个状态是持久化的，因此如果不需要更改，可以省略像 `language` 这样的参数：

In [ ]:
query = "我的名字是什么？"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob. ¿Hay algo más en lo que pueda ayudarte?


为了帮助您理解内部发生的事情，请查看 [此 LangSmith 跟踪](https://smith.langchain.com/public/15bd8589-005c-4812-b9b9-23e74ba4c3c6/r)。

## 管理对话历史

构建聊天机器人时需要理解的一个重要概念是如何管理对话历史。如果不加以管理，消息列表将无限增长并可能溢出 LLM 的上下文窗口。因此，重要的是添加一个步骤来限制您传递给模型的消息大小。

**重要的是，您需要在提示模板之前但在从消息历史加载之前的消息之后执行此操作。**

我们可以通过在提示前添加一个简单的步骤来修改 `messages` 键，然后将该新链包装在消息历史类中。

LangChain 附带了一些用于 [管理消息列表](/docs/how_to/#messages) 的内置助手。在本例中，我们将使用 [trim_messages](/docs/how_to/trim_messages/) 助手来减少我们发送给模型的消息数量。修剪器允许我们指定要保留的令牌数量，以及其他参数，例如是否始终保留系统消息以及是否允许部分消息：

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="你是一个好助手"),
    HumanMessage(content="你好！我是 Bob"),
    AIMessage(content="你好！"),
    HumanMessage(content="我喜欢香草冰淇淋"),
    AIMessage(content="不错"),
    HumanMessage(content="2 + 2 是多少"),
    AIMessage(content="4"),
    HumanMessage(content="谢谢"),
    AIMessage(content="没问题！"),
    HumanMessage(content="玩得开心吗？"),
    AIMessage(content="是的！"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

要在我们的链中使用它，我们只需要在将 `messages` 输入传递给提示之前运行修剪器。

In [24]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    # highlight-start
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    # highlight-end
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

现在，如果我们尝试询问模型我们的名字，它不会知道，因为我们修剪了聊天历史的那部分：

In [ ]:
config = {"configurable": {"thread_id": "abc567"}}
query = "我的名字是什么？"
language = "英语"

# highlight-next-line
input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. You haven't told me yet!


但是，如果我们询问最近几条消息中的信息，它会记住：

In [ ]:
config = {"configurable": {"thread_id": "abc678"}}
query = "我问了什么数学问题？"
language = "英语"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked what 2 + 2 equals.


如果您查看 LangSmith，您可以在 [LangSmith 跟踪](https://smith.langchain.com/public/04402eaa-29e6-4bb1-aa91-885b730b6c21/r) 中准确了解内部发生的情况。

## 流式传输

现在我们有了一个功能齐全的聊天机器人。然而，对于聊天机器人应用来说，一个非常重要的用户体验考虑是流式传输。LLM 有时可能需要一段时间才能响应，因此为了改善用户体验，大多数应用程序所做的一件事是流式传输每个生成的令牌。这使用户能够看到进度。

实际上，这非常容易实现！

默认情况下，我们的 LangGraph 应用中的 `.stream` 会流式传输应用步骤——在本例中，模型响应的单个步骤。设置 `stream_mode="messages"` 允许我们流式传输输出令牌：

In [ ]:
config = {"configurable": {"thread_id": "abc789"}}
query = "你好，我是 Todd，请告诉我一个笑话。"
language = "英语"

input_messages = [HumanMessage(query)]
# highlight-next-line
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    # highlight-next-line
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # 过滤到仅模型响应
        print(chunk.content, end="|")

|Hi| Todd|!| Here|’s| a| joke| for| you|:

|Why| don|’t| skeleton|s| fight| each| other|?

|Because| they| don|’t| have| the| guts|!||

## 下一步

现在您了解了如何在 LangChain 中创建聊天机器人的基础知识，您可能感兴趣的一些更高级的教程是：

- [对话式 RAG](/docs/tutorials/qa_chat_history)：在外部数据源上启用聊天机器人体验
- [代理](/docs/tutorials/agents)：构建可以执行操作的聊天机器人

如果您想深入了解具体内容，值得查看的一些内容是：

- [流式传输](/docs/how_to/streaming)：流式传输对聊天应用至关重要
- [如何添加消息历史](/docs/how_to/message_history)：深入了解与消息历史相关的所有内容
- [如何管理大型消息历史](/docs/how_to/trim_messages/)：管理大型聊天历史的更多技术
- [LangGraph 主文档](https://langchain-ai.github.io/langgraph/)：了解有关使用 LangGraph 构建的更多详细信息